# Thư viện

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install whoosh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 18.8 MB/s eta 0:00:00


In [ ]:
import os
import enum
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from whoosh import index
from whoosh.fields import Schema, TEXT
from whoosh.qparser import QueryParser
from whoosh.analysis import StemmingAnalyzer
import nltk

from os import listdir
from os.path import isfile, join
import numpy as np
from numpy.linalg import svd, norm
from numpy import dot
import os
import pickle
import math
import re
import pandas as pd
import time

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
ss = SnowballStemmer("english")

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

# Tiền xử lý dữ liệu

In [ ]:
def preprocess_text(text):
    processed_text = text.lower()
    processed_text = processed_text.replace("’", "'")
    processed_text = processed_text.replace("“", '"')
    processed_text = processed_text.replace("”", '"')
    non_words = re.compile(r"[^A-Za-z']+")
    processed_text = re.sub(non_words, ' ', processed_text)
    return processed_text

In [ ]:
def get_words_from_text(text):
    processed_text = preprocess_text(text)
    filtered_words = [word for word in processed_text.split() if word not in stopwords.words('english')]
    processed_words = []
    for i in range(len(filtered_words)):
        #processed_words.append(lemmatizer.lemmatize(filtered_words[i]))
        processed_words.append(ps.stem(filtered_words[i]))
        #processed_words.append(ss.stem(filtered_words[i]))
    return processed_words

In [ ]:
def get_text_from_file(filename):
    with open(filename, encoding='utf-8', mode='r') as f:
        text = f.read()
    return text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def load_res(filename):
    goal = []

    with open(filename, encoding='utf-8', mode='r') as f:
        related_ids = f.readlines()

    goal = [int(id) for id in related_ids[0].split() if id.strip()]

    return goal

In [ ]:
# Read Cranfield documents from a directory
cranfield_docs_dir = '/content/drive/MyDrive/CS419/Dataset/Cranfield/docs/'

# Define the path to the query folder
query_folder = "/content/drive/MyDrive/CS419/Dataset/Cranfield/queries"

# Define the path to the qrel folder
qrel_folder = "/content/drive/MyDrive/CS419/Dataset/Cranfield/qrel"

In [ ]:
# Create a list to store the document content
doc_contents = []
id_contents = []

# Read and store the document content
for filename in os.listdir(cranfield_docs_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(cranfield_docs_dir, filename)
        with open(filepath) as file:
            file_number = filename.split(".")[0]
            #content = file.read().strip()
            #document = [file_number, content]
            content = get_text_from_file(filepath)
            doc_contents.append(content)
            id_contents.append(file_number)

In [ ]:
def get_related_files(num):
    # Create the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(doc_contents)

    # Open the index
    ix = index.open_dir("cranfield_index")

    precision_result = []
    recall_result = []

    # Create a searcher
    with ix.searcher() as searcher:
        query_parser = QueryParser("content", schema=ix.schema)

        for query_filename in os.listdir(query_folder):
            total_related_files = 0
            retrieved_related_files = 0

            if query_filename.endswith(".txt"):
                query_filepath = os.path.join(query_folder, query_filename)
                with open(query_filepath) as query_file:
                    query_text = get_text_from_file(query_filepath)

                    query = query_parser.parse(query_text)

                    # Transform the query into a TF-IDF vector
                    query_vector = vectorizer.transform([query_text])

                    # Calculate cosine similarity between the query vector and the document vectors
                    similarities = cosine_similarity(query_vector, tfidf_matrix)

                    # Get the indices of the top 50 most similar documents
                    top_indices = similarities.argsort()[0][-num:]

                    # Get the corresponding document numbers
                    related_files = [id_contents[i] for i in top_indices]  # Assuming the document numbers start from 1

                    qrel_filename = f"{query_filename.split('.')[0]}.txt"
                    qrel_filepath = os.path.join(qrel_folder, qrel_filename)
                    with open(qrel_filepath) as qrel_file:
                        #related_ids = qrel_file.read().strip().split("\n")
                        related_ids = load_res(qrel_filepath)

                    # Count the number of related files
                    #total_related_files += len(related_ids)
                    #print(related_ids)
                    #print(results)
                    # Check if retrieved files are related
                    for id in related_files:
                        if int(id) in related_ids:
                            retrieved_related_files += 1

                    # Calculate precision and recall
                    precision = retrieved_related_files / len(os.listdir(query_folder))
                    recall = retrieved_related_files / len(related_ids)

                    #total_precision += precision
                    #total_recall += recall

                    precision_result.append(precision)
                    recall_result.append(recall)

                    print(f"Total Related Files: {total_related_files}")
                    print(f"Retrieved Related Files: {retrieved_related_files}")
                    print(f"Precision: {precision}")
                    print(f"Recall: {recall}")
                    print(f"Query: {query_text}")
                    print("Related Files:")
                    print(related_files)
                    print("Actual Related Files:")
                    print(related_ids)
                    print()

    return precision_result, recall_result

In [ ]:
def evaluation(precision, recall):
    PRECISION = []
    RECALL = []
    average_precision = []

    # tính AP nội suy
    temp = 0
    for i in range(len(precision) - 1, -1, -1):
        if temp >= precision[i]:
            precision[i] = temp

        elif temp < precision[i]:
            temp = precision[i]

    # tính MAP 11 điểm
    precision_list_11 = []
    points = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    for idx, i in enumerate(points):
        for j in range(len(recall)):
            if recall[j] >= i:
                precision_list_11.append(max(precision[j:]))
                break

    if len(precision_list_11) < 11:
      precision_list_11 += [0] * (11 - len(precision_list_11))

    if len(precision_list_11) != 0:
        average_precision.append(sum(precision_list_11) / 11)

    else:
        average_precision.append(0)

    PRECISION.append(precision[-1])
    RECALL.append(recall[-1])

    if len(average_precision) != 0:
        return sum(average_precision) / len(average_precision), sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)

    else:
        return 0, sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)

In [ ]:
import time

start_time = time.time()
precision, recall = get_related_files(700)
mAP, average_pre, average_rec = evaluation(precision, recall)
print('mAP = ', mAP )
print('Precision = ', average_pre )
print('Recall = ', average_rec )
print("Time: %s seconds" % (time.time() - start_time))

Total Related Files: 0
Retrieved Related Files: 5
Precision: 0.022222222222222223
Recall: 1.0
Query:  chemical  kinetic  system  applicable  hypersonic  aerodynamic problems
Related Files:
['444', '906', '941', '392', '386', '394', '414', '383', '370', '910', '898', '420', '933', '396', '404', '385', '439', '924', '934', '443', '465', '513', '509', '516', '455', '856', '907', '454', '926', '874', '450', '448', '895', '937', '881', '449', '447', '440', '482', '408', '929', '431', '380', '902', '403', '387', '916', '922', '888', '384', '936', '382', '897', '918', '915', '376', '878', '374', '911', '935', '430', '427', '389', '920', '883', '884', '900', '882', '400', '435', '433', '402', '418', '932', '901', '891', '931', '887', '923', '397', '545', '456', '493', '829', '869', '817', '498', '492', '460', '863', '475', '478', '502', '868', '489', '501', '845', '872', '871', '857', '853', '507', '487', '514', '838', '528', '809', '533', '526', '532', '820', '836', '819', '477', '834', '467'